In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd 
import numpy as np 
import os 
import sys
import binary_model

from keras import backend as K
import tensorflow as tf 
import gc
import time
from contextlib import contextmanager
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import KFold, StratifiedKFold
import matplotlib.pyplot as plt
from pandas_preprocess import PandasMinMaxScaler, PandasStandardScaler


import seaborn as sns
import warnings
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from keras.callbacks import Callback
from deep_learning_research.preprocess_toolkit import util as UT
import keras as ks
from keras.backend.tensorflow_backend import clip
from keras.layers.core import Lambda

warnings.simplefilter(action='ignore', category=FutureWarning)
pd.options.display.max_columns = 1000
pd.options.display.max_rows = 1000
tar_dir = '../../.kaggle/competitions/home-credit-default-risk/'

C:\Users\kent\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
os.listdir(tar_dir)

['application_test.csv',
 'application_train.csv',
 'bureau.csv',
 'bureau_balance.csv',
 'credit_card_balance.csv',
 'HomeCredit_columns_description.csv',
 'installments_payments.csv',
 'POS_CASH_balance.csv',
 'previous_application.csv',
 'sample_submission.csv']

### Data Preprocess Schema
- special hard code filtering 
- data preprocess 
    - data augmentation via interaction (concated, LDA)... etc 
    - data fratorization 

- first level feature extractering 
- second level models


In [ ]:
def check_null(df):
    se = df.isnull().sum() 
    se_all = len(df)
    ax_df = se.apply(lambda x: x/se_all)
    ax_df = ax_df[ax_df > 0.01]
    if len(ax_df) < 1:
        print('No Missing Data')
        return None
    ax = ax_df.plot(kind='bar',figsize=(18,2),
                    title = str(len(se) )+' Cols with '+str(len(ax_df)) + \
                    'contains null' + '(Sample Size {})'.format(len(df)),
                   )
    ax.set_xticklabels(se.index, rotation=90)
    plt.show()
check_null(temp)

### Data Proprocess with Bureau and Bureau-balance

In [3]:
bureau_balance = UT.import_data(tar_dir+'bureau_balance.csv')
temp = bureau_balance.groupby('SK_ID_BUREAU')['STATUS'].value_counts()
temp = temp.unstack().fillna(0)
temp = temp.reset_index()
bureau = UT.import_data(tar_dir+'bureau.csv')
bureau = pd.merge(bureau, temp, on='SK_ID_BUREAU', how='left')
bureau = bureau[['SK_ID_CURR', '0', '1', '2', '3', '4', '5', 'C', 'X']]
bureau = bureau.fillna(0)
del temp
del bureau_balance



Memory usage of dataframe is 624.85 MB
Memory usage after optimization is: 156.21 MB
Decreased by 75.0%
Memory usage of dataframe is 222.62 MB
Memory usage after optimization is: 78.57 MB
Decreased by 64.7%


In [4]:
bureau = bureau.groupby('SK_ID_CURR').sum().reset_index()
df = pd.read_csv('preprocessed_data_01.csv')
print(len(df), 'BEFORE')
df = pd.merge(df, bureau, on='SK_ID_CURR', how='left')
print(len(df), 'AFTER')
df_fill = df.fillna(0)

df_fill['TARGET'] = df['TARGET']
feats = [f for f in df.columns if f not in ['TARGET','SK_ID_CURR','SK_ID_BUREAU','SK_ID_PREV', 'index', 'index_x', 'index_y', 'Unnamed: 0']]
#df_fill.to_csv('preprocessed_data_02.csv', index=False)

del df

356255 BEFORE
356255 AFTER


In [5]:
target_balance = 1
if target_balance : 
    print('target balance')
    for i in range(3):
        df_fill = df_fill.append(df_fill[df_fill['TARGET']==1])
    
    a = len(df_fill[df_fill['TARGET']==1])
    b = len(df_fill[df_fill['TARGET']==0])
    print('1 [{}] samples, 0 [{}] samples, ratio : [{}]'.format(a, b, a/b)) 

target balance
1 [198600] samples, 0 [282686] samples, ratio : [0.702546288107653]


In [ ]:


# binary_model.test(test_3_df, 
#                  num_folds=5, 
#                  feats=feature4, 
#                   submission_file_name='test.csv')
# test(df_fill, 'sk001')

binary_model.kfold_lightgbm(df_fill, 
                            num_folds=3,
                            feats=feats,
                            stratified = True,)

### Feature Seectiotn 

In [ ]:
#feats = [f for f in test_2_df.columns if f not in ['TARGET','SK_ID_CURR','SK_ID_BUREAU','SK_ID_PREV', 'index', 'index_x', 'index_y']]
    
# record = binary_model.boostraping(test_2_df, X_col=feats, y_col='TARGET', models=[GradientBoostingClassifier()])

def feature_set(record, thre):
    res = []
    for k, y in record.items():
        if y > thre:
            if k not in [ 'index','index_x','index_y']:
                res.append(k)
    return res

feature4 = feature_set(record, 1)

In [ ]:
from catboost import CatBoostClassifier
feats = [f for f in test_2_df.columns if f not in ['TARGET','SK_ID_CURR','SK_ID_BUREAU','SK_ID_PREV', 'index', 'index_x', 'index_y']]
    
binary_model.kfold_trainer(df_fill, feats=feats, 
                           model=CatBoostClassifier(iterations=3000, depth=9, thread_count=2, logging_level='Silent' ))

### Train with Simple Neural Net
- Add Target Balance is important. 
- Use ROC_AUC to evaluated the final performance
- Use Output CLipping due to it is a binary classification


In [2]:
from contextlib import contextmanager
from functools import partial
from operator import itemgetter
from multiprocessing.pool import ThreadPool
import time
from typing import List, Dict

import keras as ks
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer as Tfidf
from sklearn.pipeline import make_pipeline, make_union, Pipeline
from sklearn.preprocessing import FunctionTransformer, StandardScaler
from sklearn.metrics import mean_squared_log_error
from sklearn.model_selection import KFold

def fit_predict(xs, y_train) -> np.ndarray:
    X_train, X_test = xs
    config = tf.ConfigProto(
        intra_op_parallelism_threads=1, use_per_session_threads=1, inter_op_parallelism_threads=1)
    with tf.Session(graph=tf.Graph(), config=config) as sess:
        ks.backend.set_session(sess)
        model_in = ks.Input(shape=(X_train.shape[1],), dtype='float32', sparse=True)
        out = ks.layers.Dense(204, activation='relu')(model_in)
        out = ks.layers.Dense(68, activation='relu')(out)
        out = ks.layers.Dense(68, activation='relu')(out)
        out = ks.layers.Dense(1)(out)
        model = ks.Model(model_in, out)
        model.compile(loss='binary_crossentropy', optimizer=ks.optimizers.Adam(lr=3e-3), metrics=['accuracy'])
        for i in range(3):
            with timer(f'epoch {i + 1}'):
                model.fit(x=X_train, y=y_train, batch_size=2**(11 + i), epochs=1, verbose=0)
        return model.predict(X_test)[:, 0]

C:\Users\kent\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:

class RocAucEvaluation(Callback):
    def __init__(self, validation_data=(), interval=1):
        super(Callback, self).__init__()
        self.interval = interval
        self.x_val,self.y_val = validation_data
    def on_epoch_end(self, epoch, log={}):
        if epoch % self.interval == 0:
            y_pred = self.model.predict(self.x_val, verbose=0)
            score = roc_auc_score(self.y_val, y_pred)
            print('\n ROC_AUC - epoch:%d - score:%.6f \n' % (epoch+1, score))



def binary_crossentropy_with_ranking(y_true, y_pred):
    """ Trying to combine ranking loss with numeric precision"""
    # first get the log loss like normal
    logloss = K.mean(K.binary_crossentropy(y_pred, y_true), axis=-1)
    
    # next, build a rank loss
    
    # clip the probabilities to keep stability
    y_pred_clipped = K.clip(y_pred, K.epsilon(), 1-K.epsilon())

    # translate into the raw scores before the logit
    y_pred_score = K.log(y_pred_clipped / (1 - y_pred_clipped))

    # determine what the maximum score for a zero outcome is
    y_pred_score_zerooutcome_max = K.max(y_pred_score * (y_true <1))

    # determine how much each score is above or below it
    rankloss = y_pred_score - y_pred_score_zerooutcome_max

    # only keep losses for positive outcomes
    rankloss = rankloss * y_true

    # only keep losses where the score is below the max
    rankloss = K.square(K.clip(rankloss, -100, 0))

    # average the loss for just the positive outcomes
    rankloss = K.sum(rankloss, axis=-1) / (K.sum(y_true > 0) + 1)

    # return (rankloss + 1) * logloss - an alternative to try
    return rankloss + logloss
#-----------------------------------------------------------------------------------------------------------------------------------------------------
# AUC for a binary classifier
def auc(y_true, y_pred):   
    ptas = tf.stack([binary_PTA(y_true,y_pred,k) for k in np.linspace(0, 1, 1000)],axis=0)
    pfas = tf.stack([binary_PFA(y_true,y_pred,k) for k in np.linspace(0, 1, 1000)],axis=0)
    pfas = tf.concat([tf.ones((1,)) ,pfas],axis=0)
    binSizes = -(pfas[1:]-pfas[:-1])
    s = ptas*binSizes
    return K.sum(s, axis=0)

#-----------------------------------------------------------------------------------------------------------------------------------------------------
# PFA, prob false alert for binary classifier
def binary_PFA(y_true, y_pred, threshold=K.variable(value=0.5)):
    y_pred = K.cast(y_pred >= threshold, 'float32')
    # N = total number of negative labels
    N = K.sum(1 - y_true)
    # FP = total number of false alerts, alerts from the negative class labels
    FP = K.sum(y_pred - y_pred * y_true)    
    return FP/N
#-----------------------------------------------------------------------------------------------------------------------------------------------------
# P_TA prob true alerts for binary classifier
def binary_PTA(y_true, y_pred, threshold=K.variable(value=0.5)):
    y_pred = K.cast(y_pred >= threshold, 'float32')
    # P = total number of positive labels
    P = K.sum(y_true)
    # TP = total number of correct alerts, alerts from the positive class labels
    TP = K.sum(y_pred * y_true)    
    return TP/P

@contextmanager
def timer(name):
    t0 = time.time()
    yield
    print(f'[{name}] done in {time.time() - t0:.0f} s')


NameError: name 'Callback' is not defined

In [ ]:
def fit_predict(df,  feats, num_folds=3, stratified=True, test=None):
    for col in  ['INS_PAYMENT_PERC_MAX', 'INS_PAYMENT_PERC_MEAN', 'INS_PAYMENT_PERC_SUM', 
                 'PREV_APP_CREDIT_PERC_MAX', 'PREV_APP_CREDIT_PERC_MEAN', 
                 'REF_APP_CREDIT_PERC_MAX', 'REF_APP_CREDIT_PERC_MEAN']:
        df[col] = df[col].apply(lambda x: x/(1.+abs(x)) if x < 100000 else 1.1)
        
    df = df.append(df, ignore_index=True)
    
    agent = PandasStandardScaler()
    agent.fit(df[feats])
    
    train_df = df[df['TARGET'].notnull()]
    test_df = df[df['TARGET'].isnull()]
    
    del df 
    gc.collect()
    
    train_df = agent.transform(train_df)
    test_df = agent.transform(test_df)
    
    sub_preds = np.zeros(test_df.shape[0])
    oof_preds = np.zeros(train_df.shape[0])

    
    if stratified:
        folds = StratifiedKFold(n_splits= num_folds, shuffle=True, random_state=32)
    else:
        folds = KFold(n_splits= num_folds, shuffle=True, random_state=1001)

    for n_fold, (train_idx, valid_idx) in enumerate(folds.split(train_df[feats], train_df['TARGET'])):
        train_x, train_y = train_df[feats].iloc[train_idx], train_df['TARGET'].iloc[train_idx]
        valid_x, valid_y = train_df[feats].iloc[valid_idx], train_df['TARGET'].iloc[valid_idx]
        
        config = tf.ConfigProto(
            intra_op_parallelism_threads=1, use_per_session_threads=1, inter_op_parallelism_threads=1)
        with tf.Session(graph=tf.Graph(), config=config) as sess:
            ks.backend.set_session(sess)
            model_in = ks.Input(shape=(train_x.shape[1],), dtype='float32')
            out = ks.layers.Dense(128, activation='relu')(model_in)
            out = ks.layers.Dense(64, activation='relu')(out)
            out = ks.layers.Dropout(0.2)(out)
            out = Lambda(lambda x: clip(x, min_value=0, max_value=1))(out)
            out = ks.layers.Dense(32, activation='relu')(out)
            out = ks.layers.Dense(1)(out)
            out = Lambda(lambda x: clip(x, min_value=0, max_value=1))(out)
            model = ks.Model(model_in, out)
            model.compile(loss='binary_crossentropy', 
                          optimizer=ks.optimizers.Adam(lr=1e-3), 
                          metrics=['accuracy'])
            
            RocAuc = RocAucEvaluation(validation_data=(valid_x, valid_y), interval=1)
            
            for i in range(3):
                model.fit(x=train_x, y=train_y, batch_size=200+i*48, 
                              epochs=3, verbose=1, 
                              validation_data=(valid_x, valid_y), callbacks=[RocAuc], 
                             )
            pre_test = model.predict(test_df[feats])[:, 0]
 
        sub_preds += pre_test / folds.n_splits
        if test:
            print('test')
            break
        #model.predict(X_test)[:, 0]
        
    submission_file_name = 'simple_dl_4.csv'
    test_df['TARGET'] = sub_preds
    test_df[['SK_ID_CURR', 'TARGET']].to_csv(submission_file_name, index= False)

    print('saved ', submission_file_name)

feats = [f for f in df_fill.columns if f not in ['TARGET','SK_ID_CURR','SK_ID_BUREAU','SK_ID_PREV', 'index', 'index_x', 'index_y']]

fit_predict(df_fill, feats=feats,  stratified=True, test=None)

C:\Users\kent\Anaconda3\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


Train on 641714 samples, validate on 320858 samples
Epoch 1/3
641714/641714 [==============================] - 44s 68us/step - loss: 0.6036 - acc: 0.6917 - val_loss: 0.5407 - val_acc: 0.7255

 ROC_AUC - epoch:1 - score:0.793961 

Epoch 2/3
641714/641714 [==============================] - 43s 66us/step - loss: 0.5409 - acc: 0.7267 - val_loss: 0.5294 - val_acc: 0.7334

 ROC_AUC - epoch:2 - score:0.804392 

Epoch 3/3
641714/641714 [==============================] - 42s 65us/step - loss: 0.5222 - acc: 0.7383 - val_loss: 0.5026 - val_acc: 0.7491

 ROC_AUC - epoch:3 - score:0.827117 

Train on 641714 samples, validate on 320858 samples
Epoch 1/3
641714/641714 [==============================] - 38s 59us/step - loss: 0.4900 - acc: 0.7597 - val_loss: 0.4774 - val_acc: 0.7651

 ROC_AUC - epoch:1 - score:0.844249 

Epoch 2/3
641714/641714 [==============================] - 39s 60us/step - loss: 0.4697 - acc: 0.7740 - val_loss: 0.4664 - val_acc: 0.7802

 ROC_AUC - epoch:2 - score:0.857819 

Epoch 

In [2]:
from __future__ import print_function
from __future__ import division

from sklearn.datasets import make_classification
from sklearn.cross_validation import cross_val_score
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.svm import SVC

from bayes_opt import BayesianOptimization

# Load data set and target values
data, target = make_classification(
    n_samples=1000,
    n_features=45,
    n_informative=12,
    n_redundant=7
)

def svccv(C, gamma):
    val = cross_val_score(
        SVC(C=C, gamma=gamma, random_state=2),
        data, target, 'f1', cv=2
    ).mean()

    return val

def rfccv(n_estimators, min_samples_split, max_features):
    val = cross_val_score(
        RFC(n_estimators=int(n_estimators),
            min_samples_split=int(min_samples_split),
            max_features=min(max_features, 0.999),
            random_state=2
        ),
        data, target, 'f1', cv=2
    ).mean()
    return val

if __name__ == "__main__":
    gp_params = {"alpha": 1e-5}

    svcBO = BayesianOptimization(svccv,
        {'C': (0.001, 100), 'gamma': (0.0001, 0.1)})
    svcBO.explore({'C': [0.001, 0.01, 0.1], 'gamma': [0.001, 0.01, 0.1]})

    rfcBO = BayesianOptimization(
        rfccv,
        {'n_estimators': (10, 250),
        'min_samples_split': (2, 25),
        'max_features': (0.1, 0.999)}
    )

    svcBO.maximize(n_iter=10, **gp_params)
    print('-' * 53)
    rfcBO.maximize(n_iter=10, **gp_params)

    print('-' * 53)
    print('Final Results')
    print('SVC: %f' % svcBO.res['max']['max_val'])
    print('RFC: %f' % rfcBO.res['max']['max_val'])

Initialization
-----------------------------------------------------
 Step |   Time |      Value |         C |     gamma | 
    1 | 00m00s |    0.74473 |    0.0010 |    0.0010 | 
    2 | 00m00s |    0.80703 |    0.0100 |    0.0100 | 
    3 | 00m00s |    0.67984 |    0.1000 |    0.1000 | 
    4 | 00m00s |    0.85934 |   29.8991 |    0.0396 | 
    5 | 00m00s |    0.75949 |   83.7709 |    0.0965 | 
    6 | 00m00s |    0.85730 |   16.7875 |    0.0401 | 
    7 | 00m00s |    0.92209 |    8.4303 |    0.0138 | 
    8 | 00m00s |    0.77421 |   74.4476 |    0.0851 | 
Bayesian Optimization
-----------------------------------------------------
 Step |   Time |      Value |         C |     gamma | 
    9 | 00m00s |    0.85730 |    8.1330 |    0.0401 | 
   10 | 00m00s |    0.83861 |   29.5721 |    0.0487 | 
   11 | 00m00s |    0.78944 |   16.4470 |    0.0748 | 
   12 | 00m00s |    0.77617 |   74.8205 |    0.0833 | 
   13 | 00m01s |    0.76608 |   84.1584 |    0.0903 | 
   14 | 00m01s |    0.75564 | 